In [1]:
from keras.models import Sequential
from keras.datasets import imdb
from keras.layers import Dense, Activation, Embedding, LSTM, GRU, SimpleRNN
from keras.optimizers import Adadelta
from keras.utils import np_utils, to_categorical
from keras.preprocessing import sequence
import numpy as np

from matplotlib import pyplot as plt

C:\Users\anai\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
    max_features = 20000

    (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
    maxlen=180
    X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
    X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [5]:
print(X_train.shape) # ⇛ 25000件のデータが180次元ある
print(X_train[0]) # ⇛ 0件目のデータをprint
print(y_train) # ⇛ positive, negaitveの二値のデータが入っている.
print(X_test[0]) # ⇛ 0件目のデータをprint

(25000, 180)
[  336   385    39     4   172  4536  1111    17   546    38    13   447
     4   192    50    16     6   147  2025    19    14    22     4  1920
  4613   469     4    22    71    87    12    16    43   530    38    76
    15    13  1247     4    22    17   515    17    12    16   626    18
 19193     5    62   386    12     8   316     8   106     5     4  2223
  5244    16   480    66  3785    33     4   130    12    16    38   619
     5    25   124    51    36   135    48    25  1415    33     6    22
    12   215    28    77    52     5    14   407    16    82 10311     8
     4   107   117  5952    15   256     4     2     7  3766     5   723
    36    71    43   530   476    26   400   317    46     7     4 12118
  1029    13   104    88     4   381    15   297    98    32  2071    56
    26   141     6   194  7486    18     4   226    22    21   134   476
    26   480     5   144    30  5535    18    51    36    28   224    92
    25   104     4   226    65    16  

In [4]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

for k in range(40):
  decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in X_test[k]])
  if y_test[k] == 1:
    print('positive')
  else:
    print('negative')
  print(decoded_review)

1646592/1641221 [==============================] - 2s 1us/step
negative
? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? please give this one a miss br br kristy swanson and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite lacklustre so all you madison fans give this a miss
positive
the trademark altman audio style is evident here and there i think what really makes this film work is the brilliant performance by sandy dennis it's definitely one of her darker characters but she plays it so perfectly and convincingly that it's scary michael burns does a good job as the mute young man regular altman player michael murphy ha

In [0]:
y_train = to_categorical(y_train, 2) # ラベル ⇛ one_hotベクトルへの変換.

In [0]:
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=100, embeddings_initializer='glorot_uniform', input_length=maxlen))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

lstm = model.fit(X_train, y_train, batch_size=256, epochs=5, validation_split=0.1)

In [0]:
model.summary()

In [0]:
# 実際に文章 ⇛ id化 ⇛ モデルを用いたpredictをテストする

# positive
#test_sentence = "the trademark altman audio style is evident here and there i think what really makes this film work is the brilliant performance by sandy dennis it's definitely one of her darker characters but she plays it so perfectly and convincingly that it's scary michael burns does a good job as the mute young man regular altman player michael murphy has a small part the ? moody set fits the content of the story very well in short this movie is a powerful study of loneliness sexual repression and desperation be patient ? up the atmosphere and pay attention to the wonderfully written script br br i praise robert altman this is one of his many films that deals with unconventional fascinating subject matter this film is disturbing but it's sincere and it's sure to elicit a strong emotional response from the viewer if you want to see an unusual film some might even say bizarre this is worth the time br br unfortunately it's very difficult to find in video stores you may have to buy it off the internet"

# negative
test_sentence = "this movie is very bad. don't go to watch this. so boaring. this is such a stupid movie, shit."


# 数字に変換するメソッド
def split_text(text):
  result = []
  for word in text.split():
    word = word.lower()  # 小文字しか対応していないので、小文字化
    
    # 単語が変換すればindex化, 存在しなければ0
    try:
      index = word_index[word]
    except(KeyError):
      index = 0
    result.append(index)
      
  return result

x = [split_text(test_sentence)]
x = sequence.pad_sequences(x, maxlen=maxlen)
print(x) # ⇛ paddingされたid版の文章が見れる


y = model.predict(x)
print(y) # [ネガティブの確率、ポジティブの確率]